In [12]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

In [13]:
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import networkx as nx
from collections import defaultdict
from itertools import combinations
import dwave_networkx as dnx
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import networkx as nx
from collections import defaultdict
from itertools import combinations
import time
import dwave_networkx as dnx
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from numpy import *
from math import *
from random import *

In [14]:
# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = f"amazon-braket-qbraid-jobs" # the name of the bucket
my_prefix = "mazhandufarai-40gmail-2ecom" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)
# session and device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
print('Device:', device)

Device: Device('name': DW_2000Q_6, 'arn': arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6)


In [15]:
# helper function to plot graph
def get_graph(graph, pos):
    """
    plot colored graph for given solution
    """
    # positions for all nodes
    # pos = nx.spring_layout(graph)

    # nodes
    nx.draw_networkx_nodes(graph, pos, node_size=400)

    # edges
    nx.draw_networkx_edges(graph, pos)

    # labels
    nx.draw_networkx_labels(graph, pos, font_size=20, font_family='sans-serif')

    # plot the graph
    plt.axis('off')
    #plt.savefig("figures/random_graph.png") # save as png
    plt.show();

In [16]:
# helper function to plot graph
def get_colored_graph(graph, pos, colors):
    """
    plot colored graph for given solution
    """
    
    # positions for all nodes
    # pos = nx.spring_layout(graph)
    
    colorlist = ['#377eb8', '#e41a1c']
    nx.draw_networkx(graph, pos, node_color=[colorlist[colors[int(node)]] for node in graph.nodes],
                     node_size=400, font_weight='bold', font_color='w')
    # plot the graph
    plt.axis('off');
    # plt.savefig("./figures/weighted_graph.png") # save as png

In [17]:
# Copyright 2019 D-Wave Systems, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Input values


def run_qc(row,col):
    
    if col == 0:
        progressBar.value = 11
        return []
    
    if row == 0:
        progressBar.value = 11
        return []
    
    if row == 1:
        progressBar.value = 11
        return [1 for I in range (col)]
    
    # ------- Set tunable parameters -------
    num_reads = 1000
    gamma = 80
    print("working")
    # ------- 8 up our graph -------
    vertices = row*col
    G = nx.gnp_random_graph(vertices, 1)
    print("Graph on {} nodes created with {} out of {} possible edges.".format(len(G.nodes), 
                                                                               len(G.edges), 
                                                                               len(G.nodes) * (len(G.nodes)-1) / 2))
    # for i in range len(G.edges):
    #     for j in range len(i,G.edges):
    #         if i==j:
    #             Q[(u,v)]=i
    #         elif:
    #             x=i\\4
    #             y=j\\4
    progressBar.value = 1
    # ------- Plot our graph -------
    pos = nx.spring_layout(G)
    # plot graph with labels
    get_graph(G, pos)
    progressBar.value = 2

    # ------- Set up our QUBO dictionary -------

    # Initialize our Q matrix
    Q = defaultdict(int)

    # Fill in Q matrix
    # for u, v in G.edges:
    #     Q[(u,u)] += 1
    #     Q[(v,v)] += 1
    #     Q[(u,v)] += -2

    # alpha = [1.99,-0.01,-0.01,1.99]
    # alpha = [3,5,1,7,3,2]
    # alpha = [-0.01,1.99,-0.01,3.99,-0.01,0.99]
    epsilon = 0.01
    # Minimum between alpha[0] and alpha[2]-epsilon
    # Minimum between alpha[1] and alpha[2]-epsilon
    # bh1 = [1,2]  #importance from Edgar
    bh2 = [200,100,50]



    alpha=[] #alpha = D - lambda*E
    D=[] #distance between each patient and each hospital (normalized)
    lamb=[] #Lambda is the weight between the distance and the equipment parameters
    E=[] #Equipement parameter that includes the availability of the hospital

    bh1=[] #constraints that prevent all the patients from going to the same hospital
    bh2=[] #constraints that prevent a patient from being assigned to multiple hospitals at once

    for i in range(row*col):        #Generate random lambda (weight) values between 0 and 1
        lamb.append(random.random())  
    for i in range(row*col):        #Generate random distances (normalized)
        D.append(randint(1,10))
    for i in range(row*col):        #Generate random equipment values (normalized)
        E.append(randint(1,10))
    progressBar.value = 3

    #Combine these values to obtaine alpha
    num=0 
    while num<row*col:
        alpha.append(D[num]-lamb[num]*E[num])
        num+=1
    # print("alpha",alpha)
    for i in range(row):
        bh1.append(randint(1,10))
    # print("bh1",bh1)
    for i in range(col):
        bh2.append(randint(100,200))
    # print("bh2",bh2)
    progressBar.value = 4

    # Fill the Q matrix where the alpha values are on the diagonal and the beta values are the rest
    Q = [[0 for i in range(col*row)] for j in range(row*col)]
    for i in range (row*col):
        for j in range (col*row):
            if i==j:                     #alpha on the diagonal
                Q[i][j] = alpha[i]
            else:                        #bh1 for values on the row (same hospital)
                a=(i+1)//(col+1)
                b=(j+1)//(col+1)
                if a==b:
                    Q[i][j]=bh1[a]
                else:                    #bh2 for values on the same column (same patient)
                    a=(i+1)%col
                    b=(j+1)%col
                    if a==b:
                        Q[i][j]=bh2[a]
                    else:
                        Q[i][j]=0
        # print(Q)
        # ------- Run our QUBO on the QPU -------
    progressBar.value = 5

    # Set chain strength
    chain_strength = gamma*len(G.nodes)

    # Run the QUBO on Braket
    start = time.time()

    # sampler = BraketDWaveSampler(s3_folder,"arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
    sampler = BraketDWaveSampler(s3_folder,"arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
    progressBar.value = 6
    sampler = EmbeddingComposite(sampler)
    progressBar.value = 7
    response = sampler.sample_qubo(Q, chain_strength=chain_strength, num_reads=num_reads)
    progressBar.value = 8

    # See if the best solution found is feasible, and if so print the number of cut edges.
    # sample = response.record.sample[0] #print the first sample with the best cost
    sample = response.record.sample  #print all the samples
    print(sample)
    progressBar.value = 9

    # print execution time
    end = time.time()
    print('Code execution time [sec]:', end - start)
    if type(sample) != list:
        sample=sample.tolist()

    for res in list(sample): #choose the solution that 
        m = []
        while res != []:
            m.append(res[:col])
            res = res[col:]
        m=transpose(m)

        k=0
        comp=True
        while k<len(m) and comp==True:
            if 1 not in m[k]:
                comp=False
            k+=1
        # print(m)
        if comp==True:
            m=transpose(m)
            print(m)
            break
    progressBar.value = 10

    print("\n")
    solution = ""
    sol_list = []
    for coll in range(len(m[0])):
        x = 0
        for roww in range(len(m)):
            if m[roww][coll] == 1 and x == 0:
                sol_list.append(str(roww+1))
                solution += "\n" + "Patient " + str(coll+1) + " to hospital " + str(roww+1)
                # print("Patient ", coll+1, " to hospital ", roww+1)
                x = 1
    progressBar.value = 11

    return sol_list


In [18]:
# Put the following lines in the terminal to install ipywidgets:
# pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension

# Put the following lines in the terminal to install qiskit and mqt.ddsim:
# pip install qiskit
# pip install mqt.ddsim

# Imports for the GUI of the program

import random
import ipywidgets as widgets
from IPython.display import display

# Defining the functions of the program

# Function that finds the best possible hospital for each patient

def findHospitals(searchButtonClicked):
    progressBar.value = 0
    print(int(hospitalsSlider.value))
    sol_list = run_qc(int(hospitalsSlider.value), int(patientsSlider.value))
    print(sol_list)
    # Put the function here and should return an array
    
    hospitalsDestinations = []
    
    for i in range(patientsSlider.value):
        hospitalsDestinations.append(widgets.Text(description='Hospital:'))
        hospitalsDestinations[i].value = 'Send Patient to Hospital ' + str(sol_list[i])
    
    hospitalsAccordion.children = hospitalsDestinations
    print(hospitalsDestinations)
    
    for i in range(patientsSlider.value):
        hospitalsAccordion.set_title(i, 'Patient '+ str(i+1))
    
# Designing the GUI of the program

imageFile = open('DelphiMedLogo.png', 'rb')
imageLogo = imageFile.read()
display(widgets.Image(value=imageLogo, format='png', width=350, height=175))

hospitalsSlider = widgets.IntSlider(description='Hospitals:', min=0, max=20, step=1)
display(hospitalsSlider)

patientsSlider = widgets.IntSlider(description='Patients:', min=0, max=20, step=1)
display(patientsSlider)

searchButton = widgets.Button(description='Search Hospital', button_style='Info')
searchButton.on_click(findHospitals) # Calling function to search patients
display(searchButton)

progressBar = widgets.IntProgress(min=0, max=11, value=0, step=1, bar_style='Info')
display(progressBar)

hospitalsAccordion = widgets.Accordion(children=[])
display(hospitalsAccordion)


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0b\xda\x00\x00\x02C\x08\x06\x00\x00\x00N\xca\x8d=\x…

IntSlider(value=0, description='Hospitals:', max=20)

IntSlider(value=0, description='Patients:', max=20)

Button(button_style='info', description='Search Hospital', style=ButtonStyle())

IntProgress(value=0, bar_style='info', max=11)

Accordion()